In [1]:
import data_pipeline as pipeline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall

In [12]:
PROCESSED_DIR = "../data/processed"
IMG_HEIGHT = IMG_WIDTH = 128
NUM_CHANNELS = 3
NUM_CLASSES = 2
EPOCHS = 30
BATCH_SIZE = 128

In [3]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
metrics = [BinaryAccuracy(), Precision(), Recall()]

In [5]:
def build_model(input_shape, num_classes):
    """
    build keras sequential model

    params
    ------
    input_shape: tuple
        shape of input images (height, width, channels)
    num_classes: int
        number of output classes

    returns
    -------
    model: tf.keras.Model
        compiled keras model
    """
    model = tf.keras.Sequential(
        [
            # convolutional
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            # fully connected
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ]
    )
    return model

In [6]:
def train_model(model, train_data, val_data, epochs=30):
    # compile the model
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=metrics,
    )
    # train the model
    history = model.fit(
        train_data,
        epochs=epochs,
        validation_data=val_data,
    )
    return history

In [ ]:
all_paths = pipeline.get_image_paths(PROCESSED_DIR)
train_paths = [path for path in all_paths if "train" in path]
mean, std = pipeline.calc_mean_std(train_paths)

In [ ]:
train_dir = PROCESSED_DIR + "/train"
val_dir = PROCESSED_DIR + "/val"

train_gen, val_gen = pipeline.load_data(train_dir, val_dir, mean, std)

In [ ]:
# build model
input_shape = (IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)
model = build_model(input_shape, NUM_CLASSES)

# print model summary
print("model architecture:")
model.summary()

In [ ]:
# train the model
history = train_model(model, train_gen, val_gen, EPOCHS)

In [ ]:
# plot training history
acc = history.history["binary_accuracy"]
val_acc = history.history["val_binary_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="training accuracy")
plt.plot(epochs_range, val_acc, label="validation accuracy")
plt.legend(loc="lower right")
plt.title("training and validation accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="training loss")
plt.plot(epochs_range, val_loss, label="validation loss")
plt.legend(loc="upper right")
plt.title("training and validation loss")
plt.show()

In [ ]:
# evaluate the model on the validation set after training
print("\nevaluating model on validation data after training...")
results = model.evaluate(val_gen, verbose=1)
print(f"final validation loss: {results[0]}")
print(f"final validation accuracy: {results[1]}")
print(f"final validation precision: {results[2]}")
print(f"final validation recall: {results[3]}")